In [1]:
import numpy as np
import matplotlib.pyplot as plt

#### cost/activation

In [8]:
# cost(y[1,:].reshape(1,2),y_hat[1,:].reshape(1,2), derivative=0)
def cost(y,y_hat, derivative=0):
    if derivative:
            return np.sum(y - y_hat, axis=0)
    return 1/2*np.sum(np.power(y - y_hat, 2),\
                     axis=0)

def logistic_sigmoid(x, derivative=0):
    sigm = 1/(1 + np.exp(-x))
    
    if derivative:
        return sigm*(1. - sigm)
    return sigm

t = { #dictionary for getting both the target logic values and the correlated string 
    # binary labels to represent the probabilities of 1 or 0 (first column is 0, 2nd 1)
    "AND": np.array([[1, 0],\
                     [1, 0],\
                     [1, 0],\
                     [0, 1]], dtype=np.float32)}

#### NN functions 1

In [3]:
# >>>>>>>>>>>>>>>>>>> forward_prop >>>>>>>>>>>>>>>>>>>>>>>>
def forward_prop(W1, b1, W2, b2, X):
    A1 = (X @ W1 + b1)[0]
    Z1 = logistic_sigmoid(A1) 
    A2 = (Z1 @ W2 + b2)[0]
    Y = logistic_sigmoid(A2)/np.sum(A2,axis=1)
    return A1, A2, Y

# >>>>>>>>>>>>>>>>>>> backprop >>>>>>>>>>>>>>>>>>>>>>>>>>>>
def backprop(W2, A1, A2, X, Y, t):
    if len(X.shape) < 2:
        X = X.reshape(X.shape[0], 1)

    ##  2nd layer (hidden units)
    # 1 x 2 or 4 x 2 (batch)
    step1 = (Y - t).reshape() * logistic_sigmoid(A2, derivative=1)
    step1 = step1.reshape(X.shape[0], X.shape[1])
    
    # 1 X N hidden units
    step2 = logistic_sigmoid(A1, derivative=1)
    step2 = step2.reshape(1, step2.shape[0])

     # N x 1
    step3 = W2
    
    grad_mid_layer = step1.T @ step2 @ step3 @ X.T
    
    ##  output layer (first step has been calculated already)    
    # now it's 2 x 1
    step1 = step1.T
    # 1 X N hidden units
    
    step2 = logistic_sigmoid(A1, derivative=0)
    N_no_hid_units = step2.shape[0]
    
    step2 = step2.reshape(1, N_no_hid_units)
    
    grad_output = step1 @ step2
    
    return grad_mid_layer, grad_output

# >>>>>>>>>>>>>>>>>>> init_weights_biases >>>>>>>>>>>>>>>>>>>
def init_weights_biases(NO_UNITS_L1):
    np.random.seed(1) #shown to converge for other XOR regression problem
    
    W1 = np.random.randn(2, NO_UNITS_L1)
    b1 = np.zeros((1, NO_UNITS_L1))
    W2 = np.random.randn(NO_UNITS_L1, 2) # 2 outputs, P(0) and P(1)
    b2 = np.zeros((1,2))
    return W1, b1, W2, b2

# >>>>>>>>>>>>>>>>>>> train >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#online (sample by sample) training
# all samples (X), 4 x 2, are fed
def train(X, T, NO_UNITS_L1=4, epochs=10_000,\
          learning_rate=.1, show_cost=0):
    
    converged = False
    rho = learning_rate
    Y = np.zeros((X.shape[0], 2)) # 4 x 2 always (the whole dataset is fed)
    
    W1, b1, W2, b2 = init_weights_biases(NO_UNITS_L1)
    
    if show_cost:
        print()
        
    for i in range(epochs):
        idx_done = i + 1
        for j in range(X.shape[0]):
            A1, A2, Y[j,:] = forward_prop(W1, b1, W2, b2, X[j,:].reshape(1,2))
            
            grad_mid_layer, grad_output =\
                    backprop(W2, A1, A2, X[j,:].reshape(1,2),\
                             Y[j,:].reshape(1,2), T[j,:].reshape(1,2))
            
            W1 = W1 - rho*grad_mid_layer
            W2 = W2 - rho*grad_output.T
#             b1 = b1 - rho*np.mean(grad_mid_layer)
#             b2 = b2 - rho*np.mean(grad_output)
            
            if show_cost:
                print("cost: " + str(cost(T[j,:],Y)))
        
        probabilities = np.array(np.round(Y) == T)
        
        if len(probabilities[probabilities == False]) < 1: #converged
            converged = True
            break
            
    return [W1, b1, W2, b2, X, Y, idx_done, epochs, converged, rho]


# >>>>>>>>>>>>>>>>>>> predict >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#train package is a list with [W1, b1, W2, b2, X]
# n: 0-3 selection of logical inputs; e.g. 0 == [0, 0]; 3 == [1,1]
def predict(train_pkg, n):
    A1, A2, Y = forward_prop(train_pkg[0], train_pkg[1],\
                             train_pkg[2], train_pkg[3], \
                             train_pkg[4][n,:])
    del A1, A2
    
    return Y

#### NN functions 2 (helpers)

In [12]:
# >>>>>>>>>>>>>>>>>>> train_all_gates >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def train_all_gates(X, t, no_hidden_units=2,\
                    iterations=500, rho=.01, print_cost=0):
    train_gates = {} #init dictionary

    for i in t:
        # NO_UNITS_L1 = 6  yields max matches with rho = 1 and epochs = 500
#         train_gates[i] = train(X, t[i], NO_UNITS_L1=2, epochs=500, learning_rate=1)
        train_gates[i] = train(X, t[i], NO_UNITS_L1=no_hidden_units,\
                               epochs=iterations, learning_rate=rho,\
                               show_cost=print_cost)
    return train_gates

# >>>>>>>>>>>>>>>>>>> match_all_gate_outputs >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# train_pkg_all_gates: 0, 1, 2, 3, 4, 5  [W1, b1, W2, b2, X, Y, 
            #                                 6. idx_done, 
            #                                 7. epochs
            #                                 8. converged
            #                                 9. rho
def match_all_gate_outputs(train_pkg_all_gates, t):
    
    test = range(0,4)
    
    matches_dummy = np.zeros((4,len(t)))
    matches = {}
    list_dummy = []
    
    j = 0
    for i in t:
        for k in test:
            y = predict(train_pkg_all_gates[i], k)
            a = np.round(y) == t[i][k,:] # t[i][k,:] is the target
            a = (a[0] == True) and (a[1] == True)
            matches_dummy[k, j] = a
        # for train_pkg_all_gates:
        #                                 8. converged
        #                                 6. idx_done, 
        #                                 7. epochs
        #                                 9. rho
        list_dummy = [train_pkg_all_gates[i][8], train_pkg_all_gates[i][6],\
                      train_pkg_all_gates[i][7], train_pkg_all_gates[i][9],\
                      matches_dummy[:, j]]
        matches[i] = list_dummy
        j += 1
        
    return matches

array([0.24499999, 0.        ], dtype=float32)

#### dataset / targets
X: possible inputs of a logic function.
t: dictionary with possible outputs for each logic gates. 
    4 binary ouputs to match NN's output probabilities of 0 or 1. 
    - if [p(0) p(1)] == [1 0] then probability of 0 == 1 && probability of 1 == 1

In [ ]:
X = np.array([[0,0],\
              [0,1],\
              [1,0],\
              [1,1]], dtype=np.float32)

t = { #dictionary for getting both the target logic values and the correlated string 
    # binary labels to represent the probabilities of 1 or 0 (first column is 0, 2nd 1)
    "AND": np.array([[1, 0],\
                     [1, 0],\
                     [1, 0],\
                     [0, 1]], dtype=np.float32),
    
    "NAND": np.array([[0, 1],\
                      [0, 1],\
                      [0, 1],\
                      [1, 0]], dtype=np.float32),
    
    "OR": np.array([[1, 0],\
                    [0, 1],\
                    [0, 1],\
                    [0, 1]], dtype=np.float32),
    
    "NOR": np.array([[0, 1],\
                     [1, 0],\
                     [1, 0],\
                     [1, 0]], dtype=np.float32),
    
    "XOR": np.array([[1, 0],\
                     [0, 1],\
                     [0, 1],\
                     [1, 0]], dtype=np.float32) 
}

#### running the NN

In [ ]:
train_gates = train_all_gates(X, t, no_hidden_units=8,\
                              iterations=500, rho=4 )

In [ ]:
matches = match_all_gate_outputs(train_gates, t)

### matches
matches is a Python dictionary. for ex: 

    matches["OR"]
    returns a list:
            matches["OR"][0] == boolean (converged or not, True or False)
            matches["OR"][1] == idx_done, no. of iterations to converge
            matches["OR"][2] == total iterations (epochs)
            matches["OR"][3] == learning rate used(rho)
            matches["OR"][4] == matches with target (comparing poth p(0) and p(1))

In [ ]:
for i in matches:
    print(i + ": " + str(matches[i][0]))
    print("iterations to converge: " + str(matches[i][1]))